In [1]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
import torch
import time
import wandb
wandb.login(key="f659082c2b19bf3ffaaceceb36c1e280541f6b11")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yeshu183 (yeshu183-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
base_path_train = "/kaggle/input/naturalist/inaturalist_12K/train" 
input_shape = (224,224)
id2label = {}
label2id = {}
label_list = []
X = []
y = []
for id,label in enumerate(os.listdir(base_path_train)):
    # if label[0]==".":
    #     continue
    id2label[id] = label
    label2id[label] = id
    label_list.append(label)
    label_path = os.path.join(base_path_train,label)
    for img_path in os.listdir(label_path):
        # if img_path == ".DS_Store":
        #     continue
        img = np.array(Image.open(os.path.join(label_path,img_path)))
        img.resize((*input_shape,3))
        X.append(img)
        y.append(id)
    print(f"id:{id}, Label: {label} done")
X = np.array(X).transpose(0, 3, 1, 2)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)
print(X.shape)
print(y.shape)

id:0, Label: Reptilia done
id:1, Label: Animalia done
id:2, Label: Arachnida done
id:3, Label: Amphibia done
id:4, Label: Aves done
id:5, Label: Mollusca done
id:6, Label: Fungi done
id:7, Label: Insecta done
id:8, Label: Plantae done
id:9, Label: Mammalia done
torch.Size([9999, 3, 224, 224])
torch.Size([9999])


In [3]:
base_path_test = "/kaggle/input/naturalist/inaturalist_12K/val" 
input_shape = (224,224)
X_test = []
y_test = []
for id,label in enumerate(os.listdir(base_path_test)):
    # if label[0]==".":
    #     continue
    label_path = os.path.join(base_path_test,label)
    for img_path in os.listdir(label_path):
        # if img_path == ".DS_Store":
        #     continue
        img = np.array(Image.open(os.path.join(label_path,img_path)))
        img.resize((*input_shape,3))
        X_test.append(img)
        y_test.append(id)
    print(f"Test data id:{id}, Label: {label} done")
X_test = np.array(X_test).transpose(0, 3, 1, 2)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)
print(X_test.shape)
print(y_test.shape)

Test data id:0, Label: Reptilia done
Test data id:1, Label: Animalia done
Test data id:2, Label: Arachnida done
Test data id:3, Label: Amphibia done
Test data id:4, Label: Aves done
Test data id:5, Label: Mollusca done
Test data id:6, Label: Fungi done
Test data id:7, Label: Insecta done
Test data id:8, Label: Plantae done
Test data id:9, Label: Mammalia done
torch.Size([2000, 3, 224, 224])
torch.Size([2000])


In [4]:
print(id2label)
print(label2id)
print(label_list)

{0: 'Reptilia', 1: 'Animalia', 2: 'Arachnida', 3: 'Amphibia', 4: 'Aves', 5: 'Mollusca', 6: 'Fungi', 7: 'Insecta', 8: 'Plantae', 9: 'Mammalia'}
{'Reptilia': 0, 'Animalia': 1, 'Arachnida': 2, 'Amphibia': 3, 'Aves': 4, 'Mollusca': 5, 'Fungi': 6, 'Insecta': 7, 'Plantae': 8, 'Mammalia': 9}
['Reptilia', 'Animalia', 'Arachnida', 'Amphibia', 'Aves', 'Mollusca', 'Fungi', 'Insecta', 'Plantae', 'Mammalia']


## Part-A

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
from torchsummary import summary

In [6]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)
#Getting (X,y) pairs
train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)
test_data = TensorDataset(X_test, y_test)

In [7]:
class customCNN(nn.Module):
    def __init__(self,m=16,k=3,n=256,m_factor=2,n_conv=5,n_fc=2,activation_func="ReLU",batch_norm=True,dropout=0.5):
        super().__init__()
        self.k = k # size of filters (kxk)
        self.n = n # no.of neurons in the dense layer
        self.m = m # no. of filters
        self.m_fac = m_factor # filter multiplier
        self.conv_layers = nn.ModuleList()
        self.batch_norm = batch_norm
        self.bn_layers = nn.ModuleList()
        self.fc_layers = nn.ModuleList()
        self.dropout = nn.Dropout(dropout)
        self.n_conv = n_conv
        self.n_fc = n_fc
        m = self.m
        for i in range(self.n_conv):
            if i==0:
                self.conv_layers.append(nn.Conv2d(3,int(m),self.k))
                self.bn_layers.append(nn.BatchNorm2d(int(m)))
            else:
                if m*self.m_fac>=1:
                    self.conv_layers.append(nn.Conv2d(int(m),int(m*self.m_fac),self.k))
                    self.bn_layers.append(nn.BatchNorm2d(int(m*self.m_fac)))
                    m = m*self.m_fac
                else:
                    self.m_fac = 1
                    self.conv_layers.append(nn.Conv2d(int(m),int(m*self.m_fac),self.k))
                    self.bn_layers.append(nn.BatchNorm2d(int(m*self.m_fac)))
                    m = m*self.m_fac
        for i in range(self.n_fc):
            self.fc_layers.append(nn.LazyLinear(self.n))
            
        self.maxpool = nn.MaxPool2d(2,2)
        self.fc1 = nn.LazyLinear(self.n)
        self.fc2 = nn.Linear(self.n, 10)
        activations = {
            "mish": nn.Mish(),
            "gelu": nn.GELU(),
            "silu": nn.SiLU(),
            "relu": nn.ReLU()
        }
        self.activation = activations.get(activation_func.lower(), nn.ReLU())
    def forward(self,x):
        for i in range(self.n_conv):
            x = self.conv_layers[i](x)
            if self.batch_norm:
                x = self.bn_layers[i](x)
            x = self.activation(x)
            x = self.maxpool(x)
        x = torch.flatten(x,1)
        for i in range(self.n_fc):
            x = self.fc_layers[i](x)
            x = self.dropout(x)
            x = self.activation(x)
        x = self.fc2(x)
        # output = F.softmax(x,dim=1)
        return x

In [8]:
from torch.utils.data import Dataset

class CustomImageDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img = self.X[idx]
        label = self.y[idx]

        # Convert tensor image to PIL for applying torchvision transforms
        img = transforms.ToPILImage()(img)

        if self.transform:
            img = self.transform(img)

        return img, label

In [9]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'm': {'values': [32, 64]},
        'm_factor': {'values': [1, 2, 0.5]},
        'dropout': {'values': [0.2, 0.3]},
        'activation_func': {'values': ['ReLU', 'GELU', 'SiLU', 'Mish']},
        'n_fc' : {'values': [1,2]},
        'n': {'values':[128,256]},
        'batch_norm': {'values': [True, False]},
        'batch_size': {'values': [32, 64]},
        'data_augmentation': {'values': [True, False]}
    }
}


In [10]:
def train_sweep():
    wandb.init()
    config = wandb.config

    run_name = (f"act_{config.activation_func}_bs_{config.batch_size}_m_{config.m}_n_{config.n}_nfc_{config.n_fc}"
                f"_mf_{config.m_factor}_do_{config.dropout}_bn_{config.batch_norm}_aug_{config.data_augmentation}")
    wandb.run.name = run_name

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = customCNN(
        m=config.m,
        m_factor=config.m_factor,
        n_fc=config.n_fc,
        activation_func=config.activation_func,
        batch_norm=config.batch_norm,
        dropout=config.dropout
    ).to(device)

    # Define transforms
    transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Keeps size while augmenting
        transforms.ToTensor()
    ]) if config.data_augmentation else transforms.Compose([
        transforms.Resize((224, 224)),  # Ensures all images are same size
        transforms.ToTensor()
    ])
    
    transform_val = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])


    # Use transformed datasets
    train_dataset = CustomImageDataset(X_train, y_train, transform=transform_train)
    val_dataset = CustomImageDataset(X_val, y_val, transform=transform_val)
    #test_dataset = CustomImageDataset(X_test, y_test, transform=transform_val)

    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
    #test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(5):
        epoch_start_time = time.time()
        model.train()
        running_loss, correct, total = 0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = 100 * val_correct / val_total

        epoch_time = time.time() - epoch_start_time

        wandb.log({
            'train_loss': train_loss,
            'train_accuracy': train_acc,
            'val_loss': val_loss,
            'val_accuracy': val_acc,
            'epoch_time': epoch_time
        })

        print(f"Epoch {epoch+1} | Time: {epoch_time:.2f}s | "
              f"Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.3f}, Val Acc: {val_acc:.2f}%")

In [ ]:
wandb.finish()
sweep_id = wandb.sweep(sweep_config, project="DA6401_Assignment_02")
wandb.agent(sweep_id, function=train_sweep, count=50)  # Runs 50 experiments

Create sweep with ID: 7rn82ov2
Sweep URL: https://wandb.ai/yeshu183-indian-institute-of-technology-madras/DA6401_Assignment_02/sweeps/7rn82ov2


wandb: Agent Starting Run: o0f1wfgj with config:
wandb: 	activation_func: GELU
wandb: 	batch_norm: True
wandb: 	batch_size: 64
wandb: 	data_augmentation: True
wandb: 	dropout: 0.2
wandb: 	m: 64
wandb: 	m_factor: 0.5
wandb: 	n: 256
wandb: 	n_fc: 1
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch 1 | Time: 30.93s | Train Loss: 2.251, Train Acc: 15.10% | Val Loss: 2.210, Val Acc: 18.95%
Epoch 2 | Time: 29.66s | Train Loss: 2.200, Train Acc: 18.64% | Val Loss: 2.207, Val Acc: 17.85%
Epoch 3 | Time: 29.67s | Train Loss: 2.188, Train Acc: 18.43% | Val Loss: 2.235, Val Acc: 17.50%
Epoch 4 | Time: 30.07s | Train Loss: 2.172, Train Acc: 19.28% | Val Loss: 2.186, Val Acc: 20.20%
Epoch 5 | Time: 30.54s | Train Loss: 2.161, Train Acc: 20.07% | Val Loss: 2.170, Val Acc: 19.95%


epoch_time,█▁▁▃▆
train_accuracy,▁▆▆▇█
train_loss,█▄▃▂▁
val_accuracy,▅▂▁█▇
val_loss,▅▅█▃▁
epoch_time,30.54307
train_accuracy,20.06501
train_loss,2.16145
val_accuracy,19.95
val_loss,2.17035


wandb: Agent Starting Run: qecksstq with config:
wandb: 	activation_func: ReLU
wandb: 	batch_norm: True
wandb: 	batch_size: 32
wandb: 	data_augmentation: False
wandb: 	dropout: 0.3
wandb: 	m: 64
wandb: 	m_factor: 1
wandb: 	n: 128
wandb: 	n_fc: 1


...

wandb: Agent Starting Run: ze7wlhfo with config:
wandb: 	activation_func: SiLU
wandb: 	batch_norm: False
wandb: 	batch_size: 32
wandb: 	data_augmentation: True
wandb: 	dropout: 0.2
wandb: 	m: 32
wandb: 	m_factor: 2
wandb: 	n: 128
wandb: 	n_fc: 1


Epoch 1 | Time: 24.55s | Train Loss: 2.296, Train Acc: 12.26% | Val Loss: 2.309, Val Acc: 13.65%
Epoch 2 | Time: 24.27s | Train Loss: 2.261, Train Acc: 14.71% | Val Loss: 2.263, Val Acc: 18.85%
Epoch 3 | Time: 24.50s | Train Loss: 2.244, Train Acc: 15.81% | Val Loss: 2.227, Val Acc: 17.35%
Epoch 4 | Time: 24.55s | Train Loss: 2.238, Train Acc: 15.96% | Val Loss: 2.232, Val Acc: 18.55%
Epoch 5 | Time: 24.42s | Train Loss: 2.217, Train Acc: 18.29% | Val Loss: 2.249, Val Acc: 18.00%


epoch_time,█▁▇█▅
train_accuracy,▁▄▅▅█
train_loss,█▅▃▃▁
val_accuracy,▁█▆█▇
val_loss,█▄▁▁▃
epoch_time,24.42059
train_accuracy,18.28979
train_loss,2.2171
val_accuracy,18
val_loss,2.24898


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: nhl6x2wd with config:
wandb: 	activation_func: ReLU
wandb: 	batch_norm: True
wandb: 	batch_size: 64
wandb: 	data_augmentation: False
wandb: 	dropout: 0.2
wandb: 	m: 64
wandb: 	m_factor: 1
wandb: 	n: 256
wandb: 	n_fc: 2


Epoch 1 | Time: 25.06s | Train Loss: 2.245, Train Acc: 16.06% | Val Loss: 2.199, Val Acc: 18.70%
Epoch 2 | Time: 25.03s | Train Loss: 2.203, Train Acc: 18.05% | Val Loss: 2.196, Val Acc: 17.50%


In [ ]:
from types import SimpleNamespace
def train_best_model(best_config,n_epochs=20):
    config = SimpleNamespace(**best_config)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = customCNN(
        m=config.m,
        m_factor=config.m_factor,
        n_fc=config.n_fc,
        activation_func=config.activation_func,
        batch_norm=config.batch_norm,
        dropout=config.dropout
    ).to(device)

    # Define transforms
    transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Keeps size while augmenting
        transforms.ToTensor()
    ]) if config.data_augmentation else transforms.Compose([
        transforms.Resize((224, 224)),  # Ensures all images are same size
        transforms.ToTensor()
    ])
    
    transform_val = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    # Use transformed datasets
    train_dataset = CustomImageDataset(X_train, y_train, transform=transform_train)
    val_dataset = CustomImageDataset(X_val, y_val, transform=transform_val)
    #test_dataset = CustomImageDataset(X_test, y_test, transform=transform_val)

    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
    #test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(5):
        epoch_start_time = time.time()
        model.train()
        running_loss, correct, total = 0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = 100 * val_correct / val_total

        epoch_time = time.time() - epoch_start_time

        wandb.log({
            'train_loss': train_loss,
            'train_accuracy': train_acc,
            'val_loss': val_loss,
            'val_accuracy': val_acc,
            'epoch_time': epoch_time
        })

        print(f"Epoch {epoch+1} | Time: {epoch_time:.2f}s | "
              f"Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.3f}, Val Acc: {val_acc:.2f}%")

In [ ]:
api = wandb.Api()

sweep_path = "DA6401_Assignment_02/7rn82ov2"
runs = api.runs(sweep_path)

sorted_runs = sorted(runs, key=lambda run: run.summary.get("val_accuracy", 0), reverse=True)

best_run = sorted_runs[0]
best_config = {}
print(len(sorted_runs))
print("Best Run Name:", best_run.name)
print("Best Val Accuracy:", best_run.summary["val_accuracy"])
print("Best Hyperparameters:")
for k, v in best_run.config.items():
    best_config[k] = v
print(best_config)

In [ ]:
model = train_best_model(best_config,n_epochs=30)

In [ ]:
transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
test_dataset = CustomImageDataset(X_test, y_test, transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)
model.eval()
test_loss, test_correct, test_total = 0, 0, 0
all_preds = []
all_imgs = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

        all_preds.extend(predicted.cpu())
        all_imgs.extend(inputs.cpu())

test_accuracy = 100 * test_correct / test_total
print(f" Final Test Accuracy: {test_accuracy:.2f}%")

In [ ]:
# import matplotlib.pyplot as plt

# epochs = range(1, len(train_losses) + 1)

# # Plot Loss
# plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# plt.plot(epochs, train_losses, label='Train Loss', marker='o')
# plt.plot(epochs, val_losses, label='Validation Loss', marker='o')
# plt.title('Loss over Epochs')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.grid(True)

# # Plot Accuracy
# plt.subplot(1, 2, 2)
# plt.plot(epochs, train_accuracies, label='Train Accuracy', marker='o')
# plt.plot(epochs, val_accuracies, label='Validation Accuracy', marker='o')
# plt.title('Accuracy over Epochs')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy (%)')
# plt.legend()
# plt.grid(True)

# plt.tight_layout()
# plt.show()